In [1]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [2]:
events_dict = {
    'BLUE: first_blood': 0,
    'BLUE: dragon': 1,
    'BLUE: herald': 2,
    'BLUE: first_tower_top': 3,
    'BLUE: first_tower_mid': 4,
    'BLUE: first_tower_bot': 5,
    'BLUE: second_tower_top': 6,
    'BLUE: second_tower_mid': 7,
    'BLUE: second_tower_bot': 8,
    'BLUE: third_tower_top': 9,
    'BLUE: third_tower_mid': 10,
    'BLUE: third_tower_bot': 11,
    'BLUE: inhibitor_top': 12,
    'BLUE: inhibitor_mid': 13,
    'BLUE: inhibitor_bot': 14,
    'BLUE: baron': 15,
    'BLUE: elder_dragon': 16,
    'BLUE: nexus_tower': 17,
    'BLUE: nexus': 18,
    'RED: first_blood': 19,
    'RED: dragon': 20,
    'RED: herald': 21,
    'RED: first_tower_top': 22,
    'RED: first_tower_mid': 23,
    'RED: first_tower_bot': 24,
    'RED: second_tower_top': 25,
    'RED: second_tower_mid': 26,
    'RED: second_tower_bot': 27,
    'RED: third_tower_top': 28,
    'RED: third_tower_mid': 29,
    'RED: third_tower_bot': 30,
    'RED: inhibitor_top': 31,
    'RED: inhibitor_mid': 32,
    'RED: inhibitor_bot': 33,
    'RED: baron': 34,
    'RED: elder_dragon': 35,
    'RED: nexus_tower': 36,
    'RED: nexus': 37
}

n_in = 19
df = pd.read_csv('data/clean-one-line.csv')
df = df.drop('golId', axis=1)
data = df.values.tolist()

for game in data:
    for i, s in enumerate(game):
        if s in events_dict:
            game[i] = events_dict.get(s)

sequence = [b for b in data if not(isinstance(b, float))]
sequence = [[x for x in y if not np.isnan(x)] for y in sequence]

In [3]:
def splitSequence(games, n_steps_in, n_steps_out=2):
    X, Y = [], []
    
    for sequence in games:
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out
            # check if we are beyond the sequence
            if out_end_ix > len(sequence):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
            X.append(seq_x)
            Y.append(seq_y)
	
    return X, Y

In [4]:
x_train, x_test = train_test_split(sequence,test_size=0.15,random_state=42,shuffle=False)
len(x_train)

2462

In [5]:
train, y_train = splitSequence(x_train, n_in)

In [6]:
test, y_test = splitSequence(x_test, n_in)

In [7]:
# source: https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/

# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, data):
        X1, X2, y = list(), list(), list()
        data, y_data = splitSequence(data, n_in)
        for i in range(len(data)):
                # generate source sequence
                source = data[i]
                # define padded target sequence
                target = y_data[i]
                # create padded input target sequence
                target_in = [0] + target[:-1]
                # encode
                src_encoded = to_categorical([source], num_classes=cardinality)
                tar_encoded = to_categorical([target], num_classes=cardinality)
                tar2_encoded = to_categorical([target_in], num_classes=cardinality)
                # store
                X1.append(src_encoded)
                X2.append(tar2_encoded)
                y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return X1, X2, y

def get_dataset_with_index(n_in, n_out, cardinality, data, i):
        X1, X2, y = list(), list(), list()
        data, y_data = splitSequence(data, n_in)
        # for i in range(len(data)):
        # generate source sequence
        source = data[i]
        # define padded target sequence
        target = y_data[i]
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return X1, X2, y

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source, verbose=0)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state, verbose=0)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

def has_common_data(list1, list2):
    result = False
  
    # traverse in the 1st list
    for x in list1:
  
        # traverse in the 2nd list
        for y in list2:
    
            # if one common
            if x == y:
                result = True
                return result 
                  
    return result

# configure problem
n_features = 38 + 1
n_steps_in = 19
n_steps_out = 2
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'mse', 'mae', 'mape'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, x_train)
print(X1.shape,X2.shape,y.shape)
# train model
train.fit([X1, X2], y, epochs=50)

(10373, 19, 39) (10373, 2, 39) (10373, 2, 39)
Epoch 1/50
325/325 [==============================] - 13s 15ms/step - loss: 0.0200 - accuracy: 0.3853 - mse: 0.0200 - mae: 0.0405 - mape: 20238808.0000
Epoch 2/50
325/325 [==============================] - 5s 15ms/step - loss: 0.0161 - accuracy: 0.5346 - mse: 0.0161 - mae: 0.0325 - mape: 16273542.0000
Epoch 3/50
325/325 [==============================] - 5s 15ms/step - loss: 0.0148 - accuracy: 0.5824 - mse: 0.0148 - mae: 0.0299 - mape: 14931738.0000
Epoch 4/50
325/325 [==============================] - 5s 15ms/step - loss: 0.0143 - accuracy: 0.5971 - mse: 0.0143 - mae: 0.0287 - mape: 14373505.0000
Epoch 5/50
325/325 [==============================] - 5s 15ms/step - loss: 0.0141 - accuracy: 0.6041 - mse: 0.0141 - mae: 0.0282 - mape: 14077342.0000
Epoch 6/50
325/325 [==============================] - 6s 17ms/step - loss: 0.0138 - accuracy: 0.6113 - mse: 0.0138 - mae: 0.0276 - mape: 13812908.0000
Epoch 7/50
325/325 [===========================

In [8]:
total, correct = len(x_test), 0
total

435

In [9]:
# evaluate LSTM
total, correctEqual, correctOne = len(test), 0, 0
for i in range(total):
	X1, X2, y = get_dataset_with_index(n_steps_in, n_steps_out, n_features, x_test, i)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	# print test target and predicted
	# print('y=%s, yhat=%s' % (one_hot_decode(y[0]), one_hot_decode(target)))
	if has_common_data(one_hot_decode(y[0]), one_hot_decode(target)):
		correctOne += 1
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correctEqual += 1
print('Accuracy considering equal: %.2f%%' % (float(correctEqual)/float( len(test))*100.0))
print('Accuracy considering at-least one in common: %.2f%%' % (float(correctOne)/float( len(test))*100.0))

Accuracy considering equal: 29.31%
Accuracy considering at-least one in common: 66.89%
